In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import re
import json
import numpy as np
import pandas as pd
import pickle as pkl

from collections import defaultdict 

from tensorflow import keras
from tensorflow import one_hot

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import TimeDistributed

from tensorflow.keras.optimizers import Adam

EPOCHS = 35
MAX_LEN = 50
BATCH_SIZE = 16
HIDDEN_DIM = 100
VECTOR_DIM = 100
VALID_SPLIT = 0.2

In [2]:
def get_data(path):

    X = []
    Y = []

    data = json.load(open(path))

    for dict in data:

        x = dict['input']
        y = dict['output']

        n = len(x)

        for i in range(n):
            x[i] = re.sub('@[^ ]+','<username>',x[i])
            x[i] = re.sub('http://[^ ]+','<link>',x[i])
            x[i] = re.sub('#[^ ]+','<hashtag>',x[i])

            y[i] = re.sub('@[^ ]+','<username>',y[i])
            y[i] = re.sub('http://[^ ]+','<link>',y[i])
            y[i] = re.sub('#[^ ]+','<hashtag>',y[i])

        for _ in range(MAX_LEN-n):
            x.append("")
            y.append("")
        
        X.append(x)
        Y.append(y)

    return X, Y

In [3]:
def buildDict(data):

    wordToNum = defaultdict(int)
    num = 1
    for sent in data:
        for word in sent:
            if word not in wordToNum.keys():
                wordToNum[word] = num
                num+= 1

    return wordToNum

In [4]:
def buildDictInv(wordToNum):

    numToWord = defaultdict(str)

    for key in wordToNum.keys():
        numToWord[wordToNum[key]] = key

    return numToWord

In [5]:
def tokenize(data,wordToNum):

    tokenizedData = []

    for sent in data:

        tokenizedSent = []
        for word in sent:
            tokenizedSent.append(wordToNum[word])

        tokenizedSent=np.array(tokenizedSent,dtype=float)
        tokenizedData.append(tokenizedSent)

    return np.array(tokenizedData)

In [6]:
def getModel(VOCAB_SIZE):

    model = Sequential()

    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(VOCAB_SIZE, output_dim=VECTOR_DIM, input_length=MAX_LEN, trainable=True))
    model.add(LSTM(HIDDEN_DIM, return_sequences = True))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    print(model.summary())
    
    return model

In [7]:
def defineModel():

    raw, normalized = get_data('data/data.json')

    wordToNum = buildDict(raw + normalized)
    numToWord = buildDictInv(wordToNum)
    vocab_size = len(wordToNum)+1

    model       = getModel(vocab_size)

    model.save('model.h5')
    pkl.dump(wordToNum, open('data/wordToNum.pkl', 'wb'))
    pkl.dump(numToWord, open('data/numToWord.pkl', 'wb'))

In [8]:
def train():

  model = keras.models.load_model('model.h5')
  wordToNum = pkl.load(open('data/wordToNum.pkl', 'rb'))

  raw, normalized = get_data('data/data.json')
  print("Total Samples   : ", len(raw))

  raw = tokenize(raw, wordToNum)
  normalized = tokenize(normalized, wordToNum)

  x = raw
  y = normalized
  y = y.reshape(y.shape[0], y.shape[1], 1)

  model.fit(x, y, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_split=VALID_SPLIT)

  model.save('model.h5')

In [9]:
defineModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           2222800   
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 22228)         2245028   
_________________________________________________________________
activation (Activation)      (None, 50, 22228)         0         
Total params: 4,548,228
Trainable params: 4,548,228
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
train()

Total Samples   :  4917
Epoch 1/35
246/246 [==============================] - 17s 54ms/step - loss: 3.5829 - sparse_categorical_accuracy: 0.6984 - val_loss: 2.3238 - val_sparse_categorical_accuracy: 0.6958
Epoch 2/35
246/246 [==============================] - 12s 50ms/step - loss: 2.0460 - sparse_categorical_accuracy: 0.7162 - val_loss: 2.0072 - val_sparse_categorical_accuracy: 0.7192
Epoch 3/35
246/246 [==============================] - 12s 50ms/step - loss: 1.8277 - sparse_categorical_accuracy: 0.7404 - val_loss: 1.8743 - val_sparse_categorical_accuracy: 0.7438
Epoch 4/35
246/246 [==============================] - 12s 50ms/step - loss: 1.6790 - sparse_categorical_accuracy: 0.7600 - val_loss: 1.7575 - val_sparse_categorical_accuracy: 0.7691
Epoch 5/35
246/246 [==============================] - 12s 50ms/step - loss: 1.5292 - sparse_categorical_accuracy: 0.7897 - val_loss: 1.6197 - val_sparse_categorical_accuracy: 0.8070
Epoch 6/35
246/246 [==============================] - 12s 50ms/ste